In [27]:
import requests
import json
import os
from fpdf import FPDF

# Ollama API configuration
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.2:latest"

# Directory to save generated content
OUTPUT_DIR = "book_content"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def generate_toc(subject):
    """Generate a Table of Contents for the given subject."""
    prompt = f"""
    You are an expert assistant skilled at creating book outlines.
    Given a subject your task is to create a detailed Table of Contents for a book on the given subject.
    Now generate detailed Table of Contents for a book for Subject: {subject}
    
    Structure the response as a valid JSON object with chapters and sections. 
    No other explanatory text should be present in the response.
    Output format:
    {{
        "Table_of_Contents": [
            {{
                "Title": "Chapter 1: Title of Chapter",
                "Sections": [
                    "Section 1.1: Title of Section",
                    "Section 1.2: Title of Section"
                ]
            }},
            {{
                "Title": "Chapter 2: Title of Chapter",
                "Sections": [
                    "Section 2.1: Title of Section",
                    "Section 2.2: Title of Section"
                ]
            }}
        ]
    }}
    """
    payload = {"model": MODEL_NAME, "prompt": prompt, "stream": False}
    print(prompt)
    response = requests.post(OLLAMA_URL, json=payload)
    
    if response.status_code != 200:
        raise Exception(f"Error from Ollama: {response.text}")
    
    content = response.json().get("response", "")
    try:
        if content:
            sanitized_json = re.sub(r",\s*]", "]", content)
            print(f"sanitized_json: {sanitized_json}")
            toc = json.loads(sanitized_json)
            return toc
        else:
            raise Exception(f"Valid JSON not found in response: {content}")
    except json.JSONDecodeError as e:
        raise Exception(f"Failed to parse JSON response: {content}")

def generate_section(title):
    """Generate content for a given chapter or section title."""
    prompt = f"""
    You are a knowledgeable assistant tasked with writing detailed content for a book.
    Please write content for the following chapter or section:
    Chapter or section: '{title}'
    
    The content should be comprehensive, well-structured, and professional.
    Include examples, subtopics, and technical insights where applicable.
    """
    payload = {"model": MODEL_NAME, "prompt": prompt, "stream": False}
    print(f"prompt: {prompt}")
    response = requests.post(OLLAMA_URL, json=payload)
    
    if response.status_code != 200:
        raise Exception(f"Error from Ollama: {response.text}")
    
    return response.json().get("response", "")

def save_content_to_file(filename, content):
    """Save generated content to a file."""
    with open(os.path.join(OUTPUT_DIR, filename), "w") as f:
        f.write(content)

def create_pdf_from_content(toc, output_file="Generated_Book.pdf"):
    """Generate a PDF book from the content."""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_left_margin(15)
    pdf.set_right_margin(15)
    
    # Title Page
    pdf.add_page()
    pdf.set_font("Arial", size=24, style="B")
    pdf.cell(0, 10, "Generated Book", ln=True, align="C")
    pdf.ln(20)
    
    # Table of Contents
    pdf.set_font("Arial", size=16, style="B")
    pdf.cell(0, 10, "Table of Contents", ln=True)
    pdf.ln(10)
    pdf.set_font("Arial", size=12)
    for chapter in toc["Table_of_Contents"]:
        for chapter_title, sections in chapter.items():
            pdf.cell(0, 10, chapter_title, ln=True)
            if isinstance(sections, dict):
                for section_title in sections.get("Sections", []):
                    pdf.cell(10, 10, f"  {section_title}", ln=True)
            pdf.ln(5)
    
    # Chapters
    for chapter in toc["Table_of_Contents"]:
        for chapter_title, sections in chapter.items():
            pdf.add_page()
            pdf.set_font("Arial", size=18, style="B")
            pdf.cell(0, 10, chapter_title, ln=True)
            pdf.ln(10)
            
            filename = f"{chapter_title.replace(' ', '_')}.txt"
            file_path = os.path.join(OUTPUT_DIR, filename)
            if os.path.exists(file_path):
                with open(file_path, "r") as f:
                    content = f.read()
                pdf.set_font("Arial", size=12)
                pdf.multi_cell(0, 10, content)
    
    pdf.output(output_file)

# Main Execution
if __name__ == "__main__":
    subject = "Prompt Engineering"
    print(f"Generating Table of Contents for subject: {subject}")
    
    # Step 1: Generate ToC
    toc = generate_toc(subject)
    print("Table of Contents generated successfully.")
    print(toc)
    save_content_to_file(f"toc.txt", str(toc))
    
    # Step 2: Generate content for each chapter/section
    for chapter in toc["Table_of_Contents"][:1]:
        for chapter_title, sections in chapter.items():
            if chapter_title == "Title":
                # If this is the chapter (not the "Sections" key)
                print(f"chapter_title: {chapter_title}, sections: {sections}")
                print(f"Generating content for {sections}...")
                chapter_content = generate_section(sections)  # Generate content for the chapter
                save_content_to_file(f"{chapter_title.replace(' ', '_')}.txt", chapter_content)

            if isinstance(sections, list):  # If sections exist, handle them
                for section_title in sections:
                    print(f"Generating content for {section_title}...")
                    section_content = generate_section(section_title)  # Generate content for the section
                    save_content_to_file(f"{section_title.replace(' ', '_')}.txt", section_content)
    
    # Step 3: Create PDF
    print("Merging content into a PDF book...")
    create_pdf_from_content(toc)
    print("PDF book generated successfully: Generated_Book.pdf")


Generating Table of Contents for subject: Prompt Engineering

    You are an expert assistant skilled at creating book outlines.
    Given a subject your task is to create a detailed Table of Contents for a book on the given subject.
    Now generate detailed Table of Contents for a book for Subject: Prompt Engineering
    
    Structure the response as a valid JSON object with chapters and sections. 
    No other explanatory text should be present in the response.
    Output format:
    {
        "Table_of_Contents": [
            {
                "Title": "Chapter 1: Title of Chapter",
                "Sections": [
                    "Section 1.1: Title of Section",
                    "Section 1.2: Title of Section"
                ]
            },
            {
                "Title": "Chapter 2: Title of Chapter",
                "Sections": [
                    "Section 2.1: Title of Section",
                    "Section 2.2: Title of Section"
                ]
            }

KeyboardInterrupt: 